In [1]:
import os
import pandas as pd
import numpy as np
import re

In [2]:
MAX_SEQUENCE_LENGTH = 100
MAX_VOCAB_SIZE = 20000
EMBEDDING_DIM = 100
VALIDATION_SPLIT = 0.2
BATCH_SIZE = 128
EPOCHS = 10

In [3]:
#loading in glove
word2vec = {}
with open(os.path.join('D:\\GLOVE\\glove.6B.100d.txt'), encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        vec = np.asarray(values[1:], dtype='float32')
        word2vec[word] = vec
print('Found {} word vectors.'.format(len(word2vec)))

Found 400000 word vectors.


In [4]:
file_directory = 'D://Data//Kaggle//disaster_tweets//'
train_file = os.path.join(file_directory, 'train.csv')
test_file = os.path.join(file_directory, 'test.csv')

df = pd.read_csv(train_file)
df_test = pd.read_csv(test_file)

In [5]:
df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [21]:
class df_preprocess:
    def __init__ (self, df):
        self.df = df
    
    def text_cleaner (self, text):
        text = text.lower()
        text = re.sub(r"(?:\@|https?\://)\S+", "", text)
        text = self.remove_punctuation(text)
        text = self.remove_short_words(text)
        return text
    
    def remove_punctuation (self, text):
        punctuations = "-[]#!@$%^&*()_+<>?:.,;'"
        for t in text:
            if t in punctuations:
                text = text.replace(t, "")
        return text
    
    def remove_short_words(self, text):
        return re.sub(r'\b\w{1}\b', '', text)
    
    def clean_column(self, col):
        self.df[col] = self.df[col].apply(lambda x: self.text_cleaner(x))
        return self.df

In [22]:
#Preparing the training data file
pp = df_preprocess(df)
sents = pp.clean_column('text')['text'].values
targets = df['target']

In [23]:
sents

array(['our deeds are the reason of this earthquake may allah forgive us all',
       'forest fire near la ronge sask canada',
       'all residents asked to shelter in place are being notified by officers no other evacuation or shelter in place orders are expected',
       ..., 'm194 0104 utc5km  of volcano hawaii',
       'police investigating after an bike collided with  car in little portugal bike rider suffered serious nonlife threatening injuries',
       'the latest more homes razed by northern california wildfire  abc news'],
      dtype=object)